In [1]:
import pyiron_contrib

In [2]:
from ase.lattice.cubic import Diamond

In [3]:
from pyiron_atomistics import Project, ase_to_pyiron

In [4]:
pr = Project("test")

In [5]:
pr.remove_jobs_silently(recursive=True)

100%|██████████| 1/1 [00:00<00:00, 54.66it/s]


In [6]:
job = pr.create.job.Cp2kJob("cp2k_demo")
job.structure = ase_to_pyiron(Diamond(
    directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
    symbol='Si',
    latticeconstant=5.430697500,
    size=(1, 1, 1)
))
job.run()

The job cp2k_demo was saved and received the ID: 1


In [7]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,1,finished,Si8,cp2k_demo,/cp2k_demo,/home/jovyan/,test/,2021-12-10 18:37:53.796303,2021-12-10 18:39:02.369638,68.0,pyiron@jupyter-jan-2djanssen-2dpyiron-2dcp2k-2dfzthln5b#1,cp2k,0.1,None,None


In [8]:
job.decompress()

In [9]:
job["pyiron.out"]

[' DBCSR| CPU Multiplication driver                                           XSMM\n',
 ' DBCSR| Multrec recursion limit                                              512\n',
 ' DBCSR| Multiplication stack size                                           1000\n',
 ' DBCSR| Maximum elements for images                                    UNLIMITED\n',
 ' DBCSR| Multiplicative factor virtual images                                   1\n',
 ' DBCSR| Use multiplication densification                                       T\n',
 ' DBCSR| Multiplication size stacks                                             3\n',
 ' DBCSR| Use memory pool for CPU allocation                                     F\n',
 ' DBCSR| OMP: Current number of threads                                         8\n',
 ' DBCSR| OMP: Max number of threads                                             8\n',
 ' DBCSR| Split modifier for TAS multiplication algorithm                  1.0E+00\n',
 '\n',
 '\n',
 '  **** **** ******  **  PRO